In [1]:
import pandas as pd , numpy as np 

In [2]:
pd.set_option('display.max_columns',100)

In [3]:
#1 read data

df = pd.read_csv('dummy_credit_risk_data.csv')

In [4]:
#2 making monthly pd data

monthly = (df.groupby("sourcing_month").agg(total_exposure=("ID", "count"),defaults=("dpd_90_ever", "sum")).reset_index())
monthly["pd"] = monthly["defaults"] / monthly["total_exposure"]
monthly.head()

,sourcing_month,total_exposure,defaults,pd
0,2024-01-01,10000,667,0.066700
1,2024-02-01,15000,976,0.065067
2,2024-03-01,20000,1312,0.065600
3,2024-04-01,20000,1255,0.062750
4,2024-05-01,20000,1260,0.063000


In [5]:
# 3. Fit Beta distribution
mu   = monthly["pd"].mean()
var  = monthly["pd"].var()
alpha = mu * ( (mu*(1-mu) / var) - 1 )
beta  = (1 - mu) * ( (mu*(1-mu) / var) - 1 )


In [6]:
mu, var, alpha, beta

(0.06439722222222222,
 2.6282685185185157e-06,
 1476.1724402406055,
 21446.748600462408)

In [29]:
#simualtion MC dynamic VaR

n_sims      = 100_000
lgd         = 0.25
ead_vector  = monthly["total_exposure"].values   # length = M months
M           = len(ead_vector)

# 1. Simulate PDs matrix shape=(n_sims, M)
sim_pds_matrix = np.random.beta(alpha, beta, size=(n_sims, M))

# 2. Tile exposures to match shape
ead_matrix = np.tile(ead_vector, (n_sims, 1))

# 3. Compute losses matrix shape=(n_sims, M)
sim_losses_matrix = sim_pds_matrix * ead_matrix * lgd

# 4. Extract risk metrics per month
var99_per_month = np.percentile(sim_losses_matrix, 99, axis=0)
mean_loss_per_month = sim_losses_matrix.mean(axis=0)

# ensure sourcing_month is datetime
monthly["sourcing_month"] = pd.to_datetime(monthly["sourcing_month"])

# now m is a Timestamp, so f"{m:%Y-%m}" works
for i, m in enumerate(monthly["sourcing_month"]):
    print(f"{m:%Y-%m}: 99% VaR = {var99_per_month[i]:.2f}")



2024-01: 99% VaR = 170.58
2024-02: 99% VaR = 255.91
2024-03: 99% VaR = 341.25
2024-04: 99% VaR = 341.07
2024-05: 99% VaR = 341.09
2024-06: 99% VaR = 255.75


In [36]:
monthly

,sourcing_month,total_exposure,defaults,pd
0,2024-01-01,10000,667,0.066700
1,2024-02-01,15000,976,0.065067
2,2024-03-01,20000,1312,0.065600
3,2024-04-01,20000,1255,0.062750
4,2024-05-01,20000,1260,0.063000
5,2024-06-01,15000,949,0.063267


In [32]:
170.52/0.25, 255.87/0.25, 341.1/0.25, 341.15/0.25, 341.12/0.25, 255.87/0.25

(682.08, 1023.48, 1364.4, 1364.6, 1364.48, 1023.48)

In [33]:
var99_per_month

array([170.58103106, 255.91427891, 341.24898413, 341.07403568,
       341.08842049, 255.74792372])

In [34]:
mean_loss_per_month

array([160.99493401, 241.50340127, 322.01067962, 322.00513239,
       321.99419935, 241.47319533])

In [35]:
# difference of both gives us unexpected credit loss

In [39]:
#### forecasting 6 month PD with using IID


from statsmodels.tsa.ar_model import AutoReg

# parameters
n_sims      = 50_000
H           = 3                # six-month horizon
lgd         = 0.25

# historical exposures (array of length M)
ead_hist    = monthly["total_exposure"].values

# 1. Fit AR(1) on exposures
model_ead   = AutoReg(ead_hist, lags=1, old_names=False).fit()
# 2. Forecast next H exposures
ead_forecast = model_ead.predict(start=len(ead_hist), end=len(ead_hist)+H-1)

# 3. Simulate PDs matrix (n_sims × H) from your Beta(alpha,beta)
sim_pds_matrix = np.random.beta(alpha, beta, size=(n_sims, H))

# 4. Build EAD matrix by repeating the H-vector for each sim
ead_matrix      = np.tile(ead_forecast, (n_sims, 1))

# 5. Compute monthly loss matrix and then total loss per sim
loss_matrix     = sim_pds_matrix * ead_matrix * lgd
total_losses    = loss_matrix.sum(axis=1)

# 6. 6-month VaR at 99%
var_99_6m       = np.percentile(total_losses, 99)

# display
for i, e in enumerate(ead_forecast):
    print(f"Month {i+1} forecast EAD = {e:.0f}")
print(f"3-month 99% VaR = {var_99_6m:.2f} customer-units")

Month 1 forecast EAD = 17500
Month 2 forecast EAD = 18125
Month 3 forecast EAD = 18281
3-month 99% VaR = 897.29 customer-units


In [41]:
#### forecasting 6 month PD with using AR

# 1. Fit AR(1) on historical pd
model_pd   = AutoReg(monthly["pd"].values, lags=1, old_names=False).fit()
# 2. Forecast pd for next H months
pd_forecast = model_pd.predict(start=len(monthly), end=len(monthly)+H-1)

# 3. Build pd_matrix by repeating for each sim
pd_matrix   = np.tile(pd_forecast, (n_sims, 1))

# 4. loss_matrix = pd_matrix * ead_matrix * lgd (no Beta sampling)
loss_matrix = pd_matrix * ead_matrix * lgd
total_losses= loss_matrix.sum(axis=1)
var_99_6m   = np.percentile(total_losses, 99)

# display
for i, e in enumerate(ead_forecast):
    print(f"Month {i+1} forecast EAD = {e:.0f}")
print(f"3-month 99% VaR = {var_99_6m:.2f} customer-units")

Month 1 forecast EAD = 17500
Month 2 forecast EAD = 18125
Month 3 forecast EAD = 18281
3-month 99% VaR = 854.65 customer-units


In [43]:
854.65/0.25 ##overall max customer defaults at 99% percentile

3418.6

In [44]:
3418.6/3 ## per month buffer at 99% percentile

1139.5333333333333